In [1]:
import multiprocessing
from GCForest import gcForest
import pandas as pd
import numpy as np
import pickle 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import Imputer
import warnings
warnings.filterwarnings("ignore")
#用来计算程序运行时间
import datetime
starttime = datetime.datetime.now()

In [2]:
#读取Y
all_df_y = pickle.load(open("./dump_file/all_df_y","rb"))
all_df_y['2cArousal'] = 0
all_df_y['2cArousal'][all_df_y['valence'] >= 5] = 1
all_df_y['2cValence'] = 0
all_df_y['2cValence'][all_df_y['valence'] >= 5] = 1
print(all_df_y.head(5))

#读取1个通道的PPG数据，每个通道包含32×40=1280个信号样本
#每个样本向量大小为8064点（63s*128Hz）
all_df_PPG_x = pickle.load(open("./dump_file/all_df_PPG_x","rb"))
y = all_df_y[['2cValence']]
#y = all_df_y[['2cArousal']]
for seed in [0,100,200,300,400,500,600,700,800,900]:
    print("[seed:{}]****************************************************".format(seed))
    xTrainIdx = pickle.load(open("./dump_file/xTrainIdx_{}".format(seed),"rb"))
    xTestIdx = pickle.load(open("./dump_file/xTestIdx_{}".format(seed),"rb"))
    y_tr = y.loc[xTrainIdx]
    y_te = y.loc[xTestIdx]

    PPGTrainSet = all_df_PPG_x.loc[xTrainIdx]
    PPGTestSet = all_df_PPG_x.loc[xTestIdx]

    myWindowsSize = 256
    myStrideSize = 64
    gcf = gcForest(shape_1X=8064, window=myWindowsSize, stride=myStrideSize,tolerance=0.0,n_cascadeRF=1, 
                   min_samples_mgs=0.1, min_samples_cascade=0.1,n_jobs=19)
    if True:     
        print("PPG running multi-grain scan")
        xTrain,yTrain = PPGTrainSet.values,y_tr.values
        xTest = PPGTestSet.values   
        PPG_mgsTrainVector = gcf.mg_scanning(xTrain,yTrain)
        PPG_mgsTestVector = gcf.mg_scanning(xTest)
        filePath = "./dump_file_V2/PPG_mgsTrainVector_{}_{}_{}".format(myWindowsSize,myStrideSize,seed)
        pickle.dump(PPG_mgsTrainVector,open(filePath,"wb"))
        filePath = "./dump_file_V2/PPG_mgsTestVector_{}_{}_{}".format(myWindowsSize,myStrideSize,seed)
        pickle.dump(PPG_mgsTestVector,open(filePath,"wb"))
    else:
        filePath = "./dump_file_V2/PPG_mgsTrainVector_{}_{}".format(myWindowsSize,myStrideSize)
        PPG_mgsTrainVector = pickle.load(open(filePath,"rb"))
        filePath = "./dump_file_V2/PPG_mgsTestVector_{}_{}".format(myWindowsSize,myStrideSize)
        PPG_mgsTestVector = pickle.load(open(filePath,"rb"))

    X_tr_vector = PPG_mgsTrainVector
    X_te_vector = PPG_mgsTestVector
    print(X_tr_vector.shape)
    print(X_te_vector.shape)

    #有缺失值，填充下
    X_tr_vector_fillna= pd.DataFrame(X_tr_vector).fillna(0).values
    X_te_vector_fillna= pd.DataFrame(X_te_vector).fillna(0).values

    _ = gcf.cascade_forest(X_tr_vector_fillna, y_tr)

    pred_proba = gcf.cascade_forest(X_te_vector_fillna)
    PPG = np.mean(pred_proba, axis=0)
    preds = np.argmax(PPG, axis=1)
    print("ACC",accuracy_score(y_true=y_te, y_pred=preds))
    print("F1",f1_score(y_true=y_te, y_pred=preds))
    print("Recal",recall_score(y_true=y_te, y_pred=preds))
    print("Precision",precision_score(y_true=y_te, y_pred=preds))

       valence  arousal  dominance  liking  2cArousal  2cValence
s01_0     7.71     7.60       6.90    7.83          1          1
s01_1     8.10     7.31       7.28    8.47          1          1
s01_2     8.58     7.54       9.00    7.08          1          1
s01_3     4.94     6.01       6.12    8.06          0          0
s01_4     6.96     3.92       7.19    6.05          1          1
[seed:0]****************************************************
BVP running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
(896, 492)
(384, 492)
Adding/Training Layer, n_layer=1
Layer validation accuracy = 0.6222222222222222
Adding/Training Layer, n_layer=2
Layer validation accuracy = 0.6222222222222222
ACC 0.6692708333333334
F1 0.7590132827324478
Recal 0.9216589861751152
Precision 0.6451612903225806
[seed:100]****************************************************
BVP running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
(896,